# **Personal Trainer**

Bu projede antremanda yapılan tekrar sayıları sayılacaktır.

İki video üzerinden çalışma gerçekleştirilecektir.

Birinci videoda kaç şınav çekildiği hesaplanırken el, dirsek, omuz arasındaki açıdan yararlanılacaktır.

İkinci videoda bacak çekme hareketinde kaç tekrar olduğunun anlaşılması için ayak bileği, diz, kalça arasındaki açıdan yararlanılacaktır.

3 nokta kullanılarak açı hesabı yapılacaktır.

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import math

In [2]:
media_pipe_Pose = mp.solutions.pose

pose modülü çağırılarak media_pipe_Pose objesi yaratılmıştır.

In [3]:
pose = media_pipe_Pose.Pose()

media_pipe_Pose objesinden Pose methodu pose değişkenine aktarılmıştır.

In [4]:
media_pipe_draw = mp.solutions.drawing_utils

In [5]:
#video capture
capture = cv2.VideoCapture(r"C:\Users\ilhan\Desktop\video2.mp4")

drawing_utils ile insan üzerindeki iskelet görüntüsü elde edilir.

In [6]:
def findAngle(frame, p1, p2, p3, lm_list, draw = True):
    x1, y1 = lm_list[p1][1:] #lm_list 3 nokta verir (zxy). 1'den başlandığında x,y alınır. 
    x2, y2 = lm_list[p2][1:]
    x3, y3 = lm_list[p3][1:]
    
    #açı hesaplama
    angle = math.degrees(math.atan2(y3 - y2, x3 - x2) - math.atan2(y1 - y2, x1 - x2))
    
    if angle < 0:
        angle += 360
        
    if draw:
        
        cv2.line(frame, (x1,y1), (x2,y2), (0,0,255), 3)
        cv2.line(frame, (x3,y3), (x2,y2), (0,0,255), 3)
        #dikkat edilmesi gereken nokta, x2 ortada olmalıdır.
        
        #içi dolu circle
        cv2.circle(frame, (x1,y1), 10, (0,255,255), cv2.FILLED)
        cv2.circle(frame, (x2,y2), 10, (0,255,255), cv2.FILLED)
        cv2.circle(frame, (x3,y3), 10, (0,255,255), cv2.FILLED)
        
        #içi boş circle
        cv2.circle(frame, (x1,y1), 15, (0,255,255), cv2.FILLED)
        cv2.circle(frame, (x2,y2), 15, (0,255,255), cv2.FILLED)
        cv2.circle(frame, (x3,y3), 15, (0,255,255), cv2.FILLED)
        
        cv2.putText(frame, str(int(angle)), (x2 - 40, y2 - 40),cv2.FONT_HERSHEY_PLAIN, 2, (0,255,255), 2)
        
    return angle

In [7]:
dir = 0   #hareketin bitip bitmediği kontrol edilecektir.
count = 0 #hareketin sayımı gerçekleştirilir.

#video read
while True:
    ret, frame = capture.read()
    
    if ret == True:
        
        #converting bgr to rgb format
        imgRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        result = pose.process(imgRGB) #mediapipe'ta görüntü işlenir.
        #print(result.pose_landmarks)
        #result, kamerada vücut göründüğünde, eklemlerin x,y,z koordinatlarını gösterir.
        
        lm_list = []
        
        if result.pose_landmarks:
            media_pipe_draw.draw_landmarks(frame, result.pose_landmarks, media_pipe_Pose.POSE_CONNECTIONS)
            #draw_landmarks parametresi ile iskelet yapısı çizdirilir.
            
            for id, lm in enumerate(result.pose_landmarks.landmark):
                h, w, color = frame.shape
                cx, cy = int(lm.x*w), int(lm.y*h)
                lm_list.append([id,cx,cy])
        #print(lm_list)
        
        if lm_list != 0:
            
            #şınav
            
            #angle = findAngle(frame, 11, 13, 15, lm_list)
            #per = np.interp(angle, (185, 245), (0, 100))
            #print(angle)
            
            #lm_list içerisindeki ilgili noktaların koordinatları bulunacaktır.
            #bu koordinatlar kullanılarak açı hesaplanacaktır.
            
            #bacak çekme
            
            angle = findAngle(frame, 23, 25, 27, lm_list)
            per = np.interp(angle, (77, 145), (0, 100))
            
            if per == 100:
                if dir == 0:
                    count += 0.5
                    dir = 1
            if per == 0:
                if dir == 1:
                    count += 0.5
                    dir = 0  
                
            #print(count)
                    
            cv2.putText(frame, str(int(count)), (45,125), cv2.FONT_HERSHEY_PLAIN, 10, (255,0,0), 10)
        
        cv2.imshow('Pose Estimation',frame)
        
        if cv2.waitKey(50) & 0xFF == ord('q'):
            break

    else:
        break
        
capture.release()
cv2.destroyAllWindows()